In [44]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' # to print multiple outputs from the same cell
import math
import utils
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from operator import index
from collections import defaultdict
from scipy.stats import pearsonr
from datetime import datetime

In [45]:
users_df = pd.read_csv("dataset/users.csv")

# Data Understanding and Preparation

## Users Data

In [52]:
users_df.info(verbose=True, show_counts=True, memory_usage= "deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11505 entries, 0 to 11507
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              11505 non-null  int64         
 1   name            11504 non-null  object        
 2   lang            11505 non-null  object        
 3   bot             11505 non-null  int64         
 4   created_at      11505 non-null  datetime64[ns]
 5   statuses_count  11106 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 2.1 MB


### Attribute type and quality

In the **user** dataset there are 6 columns:

1. The id **column** seems to be ok, all values are integer and there are not null values, we have to check possible duplicates
 
2. We have 1 null value in the **name** column, we also assume that the name could be a strin, a numer or a special character, the names are not necessary unique, but maybe is intresting to check the frequency distribution.

3. In the **lang** column we don't have null values, but we have to check whether there are problems in the pattern used to express the language, we expect a categorical attribute 

4. The **bot** column is numerical as expected (binary), we have to check whether all the numbers are 0 or 1

5. The attribute **created_at** has no null values, but we have to check the correctness of the date, both sintactic and semantic (not too far in the past or in the future)

6. The **status_count** column has 399 of null values, in the non-null values there would semm to be unexpected float

Let's verify the uniqueness of ids and names

In [47]:
print("Number of total IDs:", len(users_df["id"]))
print("Number of unique IDs:", len(pd.unique(users_df["id"])))
print("Number of total names:", len(users_df["name"]))
print("Number of unique names:", len(pd.unique(users_df["name"])))

Number of total IDs: 11508
Number of unique IDs: 11508
Number of total names: 11508
Number of unique names: 11361


All ids are unique!

As said before one name is null. 

It will be interesting to check if the duplicates names are often the same

Now let's check the lang column

In [48]:
pd.unique(users_df["lang"])  

array(['en', 'it', 'fr', 'ru', 'es', 'tr', 'en-gb', 'pt', 'nl', 'id',
       'zh-tw', 'ja', 'de', 'ko', 'en-AU', 'da', 'ar', 'en-GB',
       'Select Language...', 'zh-TW', 'zh-cn', 'pl', 'el', 'fil', 'sv',
       'xx-lc'], dtype=object)

The "lang" field is composed of [IETF language codes](https://en.wikipedia.org/wiki/IETF_language_tag). By selecting only the unique values it's possible to see that there are some erroneous values:
* "Select Language..." and "xx-lc" seems to be **default values**
* other values are not properly correct (e.g. "zh-cn" instead of "zh-CN")

In [49]:
utils.repair_lang_attribute(users_df)
pd.unique(users_df["lang"])

array(['en', 'it', 'fr', 'ru', 'es', 'tr', 'en-GB', 'pt', 'nl', 'id',
       'zh-TW', 'ja', 'de', 'ko', 'en-AU', 'da', 'ar', 'zh-CN', 'pl',
       'el', 'fil-PH', 'sv'], dtype=object)

The bot attribute is perfectly as expected, all non-null binary values

In [50]:
pd.unique(users_df["bot"])

array([1, 0])

We observe that the created_at coloumn is recognized by pandas as an object, and not as a datetime as we would expect from this attribute. Clean created_at field, by converting string to datetime

In [51]:
# parsing string to datetime obj
users_df["created_at"] = pd.to_datetime(users_df["created_at"])

This should be the end of the data understanding and cleaning of the users database

--------------------------------------------------------------------------------------------------

 and checks if all the tweets were created after the first tweet published on twitter (so we don't have something strange like a tweet created in 01-01-1990)

In [ ]:


# checks if all the tweets were created after the first tweet published on twitter (so we don't have something strange like a tweet created in 01-01-1990)
twitter_first_tweet_datetime = datetime(2006,3,21,12,50,0)
#string_to_datetime = lambda string: datetime.strptime(string, expected_format)
published_after_twitter_first_tweet = lambda x: x > twitter_first_tweet_datetime
all(map(published_after_twitter_first_tweet, users_df["created_at"]))


we could do a similar operation for the names, but people with the same name is not an error. For names it is more interesting to know how many values are missing in total.

In [ ]:
def my_isnan(a):
    return a != a

def my_isempty(a):
    if a == "":
        return True


number_of_total_names = len(users_df["name"])
not_empty_or_missing_names = []
empty_or_missing_names = []
names_with_only_spaces = []

# iterate over all names looking for errors
for value in users_df["name"]:
    if my_isnan(value) or my_isempty(value): # name is nan or is_empty string
        #print(users_df["name"])
        #print(users_df["name"].index(value))
        empty_or_missing_names.append(value)
    if str(value).strip() == "":
            names_with_only_spaces.append(value)
            #users_df["name"].drop(index=value)
    elif not(my_isnan(value) or my_isempty(value)):
        #print(users_df["name"])
        #print(users_df["name"].index(value))
        not_empty_or_missing_names.append(value)
print(f" Number of total names = {number_of_total_names} vs total name values that are not NA or empty = {len(not_empty_or_missing_names)}")

"""
for value in users_df["name"]:
    if my_isnan(value) or my_isempty(value):
        #print(users_df["name"])
        #print(users_df["name"].index(value))
        empty_or_missing_names.append(value)
        #users_df["name"].drop(index=value)
    #else:
    #    not_empty_or_missing_names.append(value)
"""
print(f" Number of total names = {number_of_total_names} vs total name values that are NA or empty = {len(empty_or_missing_names)}")

#print(len(names_with_only_spaces))
#print(empty_or_missing_names)
#print(not_empty_or_missing_names)
#array_of_nan = np.isnan(users_df["name"])

#users_df["name"][1012]
#users_df["name"][0]
#empty_or_missing_names


Since wrong values are just the 0.02% of the number of rows they are just dropped, while the other values are mapped to the correct ones.

In [ ]:
utils.repair_lang_attribute(users_df)

bot: should be yes or no; maybe there are wrong values?

"created at" should be a time; check if there are type error or if time value is strange (e.g. tweet made before twitter release, which was march 21 2006)

In [ ]:
# checks if all the datetime strings are in the valid format (YY-mm-dd H:M:S)
#expected_format = "%Y-%m-%d %H:%M:%S"
#is_datetime_format_correct = lambda x: utils.is_datetime_format_correct(x, expected_format)
#all(map(is_datetime_format_correct, users_df["created_at"]))



In [ ]:
#tweets_df = pd.read_csv("dataset/tweets.csv", usecols=["id", "user_id"]) # read only this two colums (saving space)
tweets_df = pd.read_csv("dataset/tweets.csv")

### Distribution of variables and statistics
Let's study them!

In [ ]:
users_df.hist(
    column=["statuses_count"], 
    log=True, 
    bins=utils.get_sturges_bins(len(users_df["statuses_count"]))
)

users_df.hist(
    column=["statuses_count"], 
    by="bot", 
    log=True,
    bins=utils.get_sturges_bins(len(users_df["statuses_count"])) #FIX THIS: USES ALL THE SAMPLES, NOT JUST THE BOTS AND THE USERS
)

In [ ]:
langs = pd.unique(users_df["lang"]) 
bot_freqs = []
user_freqs = []
for lang in langs:
    user_freqs.append(len(users_df.query(f"lang == '{lang}' & bot == 0")))
    bot_freqs.append(len(users_df.query(f"lang == '{lang}' & bot == 1")))
langs_df = pd.DataFrame({"lang": langs, "bot_freqs": bot_freqs, "user_freqs": user_freqs})
langs_df.plot.bar(x="lang", logy=True)

## Tweets data

### Assessing data quality

In [ ]:
tweets_df.info(verbose=True, show_counts=True, memory_usage= "deep")

In [ ]:
tweets_df.isnull().any() 

keeping only the tweets with user_id in user dataset

In [ ]:
user_ids = users_df["id"].values
user_ids.dtype
parsed_user_ids = pd.to_numeric(tweets_df["user_id"], errors="coerce")
parsed_user_ids.dtype
tweets_df[parsed_user_ids.isin(user_ids)]

clean id field by first removing nan values (just 2), then tring to cast to int and removing duplicates

In [ ]:
#tweets_df.dropna(subset=["id"], inplace=True)

# removing not numeric strings
#pd.to_numeric(tweets_df["id"])

#tweets_df["id"].isin(users)

From the method above we observe that all our atributes except for "created_at" have one or more elements with null values.

In [ ]:
"""
wrong_ids = []
for (i,k) in enumerate(df["id"]):
    if not isinstance(k, str) or not k.isnumeric():
        wrong_ids.append(i)
print(len(wrong_ids)/len(df["id"]))
""" 

### Distribution of variables and statistics
Let's study them!

In [ ]:
# should we use sturgen rule for number of bins?

# give error: ValueError: hist method requires numerical or datetime columns, nothing to plot.
#tweets_df.hist(column=["reply_count","retweet_count", "favorite_count", "num_hashtags", "num_urls", "num_mentions"])

In [ ]:
tweets_df.describe()

### Variables trasformations (?)

### Pairwise correlations